In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True, dotenv_path="./.env")
my_api_key = os.getenv("OPENAI_API_KEY")
dialpad_token =   os.getenv("DAILPAD_TOKEN")

client = OpenAI(api_key=my_api_key)

import base64

image_path1 = "data/test.png" 

In [2]:
import json
import base64

# Encode image as base64
with open(image_path1, "rb") as f:
    image1_base64 = base64.b64encode(f.read()).decode("utf-8")

mime_type = "image/png"

# Request extraction from image
response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {
            "role": "system",
            "content": "You extract and summarize information from invoices or forms."
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "You are a traffic ticketing officer extracting the license_plate, event_date, ticket_number, violation, fine_penalties, event_time, location_of_event in this image as a JSON object:Use the as exact column names in the json. Be Extra careful with the details when reading the letters especially the license_plate and time."},
                {"type": "image_url", "image_url": {"url": f"data:{mime_type};base64,{image1_base64}"}}
            ]
        }
    ],
)

# Extract model output
extracted_text = response.choices[0].message.content
print("🧾 Extracted Info from Image:\n")
print(extracted_text)

# Parse JSON string into dictionary
extracted_info = json.loads(extracted_text)

# Access fields
license_plate = extracted_info.get("license_plate")[-7:]
date = extracted_info.get("event_date")
ticket_no = extracted_info.get("ticket_number")
violation = extracted_info.get("violation")
fine_penalties = extracted_info.get("fine_penalties")
time = extracted_info.get("event_time")
location =  extracted_info.get("location_of_event")
date_str      = date
time_str      = time

🧾 Extracted Info from Image:

{
  "license_plate": "CA27472Y3",
  "event_date": "07/02/25",
  "ticket_number": "2011638297",
  "violation": "BUS ZONE",
  "fine_penalties": "$406.00",
  "event_time": "06:47 AM",
  "location_of_event": "SB GRAND AVE @ 3RD S"
}


In [3]:
# Install missing package in the notebook environment
%pip install pytz

import pyodbc
from datetime import datetime, timedelta
import os
import pytz
from dotenv import load_dotenv

# ── 0) Load environment variables ──────────────────────────
load_dotenv()
db_password = os.getenv("DB1_PASSWORD")
datatable   = "uber_ev_trip_activity"

# ── 1) Connect to SQL Server ───────────────────────────────
driver   = "{ODBC Driver 17 for SQL Server}"
server   = "towermobility.database.windows.net"
database = "tower_mobility_db1"
username = "CloudSA8353f88b"
password = db_password

conn_str = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
print("✅ Connected to Azure SQL")

# ── 2) Assume extracted_info exists ────────────────────────
# (from your image extraction step)
# Example fallback to avoid NameError:

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Connected to Azure SQL


In [4]:
from datetime import datetime, timedelta
import pytz
import pyodbc

# ── Inputs ──
date_str       = date
time_str       = time
license_plate  = extracted_info.get("license_plate")
datatable      = "dbo.uber_ev_trip_activity"

# ── 3) Process license plate (last 7 characters) ──
if license_plate:
    license_plate = license_plate[-7:]
else:
    raise ValueError("License plate is missing")

# ── 4) Convert ticket datetime from CA → UTC ──
la_tz = pytz.timezone("America/Los_Angeles")
utc_tz = pytz.utc

try:
    local_ticket_dt = la_tz.localize(datetime.strptime(f"{date_str} {time_str}", "%m/%d/%Y %I:%M %p"))
except ValueError:
    local_ticket_dt = la_tz.localize(datetime.strptime(f"{date_str} {time_str}", "%m/%d/%y %I:%M %p"))

ticket_datetime_utc = local_ticket_dt.astimezone(utc_tz)
ticket_date_utc = ticket_datetime_utc.date()

print("🕒 Local CA time:", local_ticket_dt)
print("🌍 UTC for SQL query:", ticket_datetime_utc)

# ── 5) Query driver ──
query = f"""
WITH trips_with_next AS (
    SELECT driver_uuid,
           trip_request_time,
           LEAD(trip_request_time) OVER (
               PARTITION BY license_plate
               ORDER BY trip_request_time
           ) AS next_trip_time
    FROM {datatable}
    WHERE license_plate = ?
)
SELECT TOP 1 driver_uuid
FROM trips_with_next
WHERE trip_request_time <= ?
  AND (next_trip_time IS NULL OR next_trip_time > ?)
ORDER BY trip_request_time DESC
"""

cursor.execute(query, license_plate, ticket_datetime_utc, ticket_datetime_utc)
row = cursor.fetchone()
driver_uuid = row[0] if row else None

print("driver_uuid:", driver_uuid)




🕒 Local CA time: 2025-07-02 06:47:00-07:00
🌍 UTC for SQL query: 2025-07-02 13:47:00+00:00
driver_uuid: A5C81082-D4CF-4D02-9005-98D18F957B33


In [5]:
employee_lookup = f"""
SELECT TOP 1 
    firstName, LastName, homeAddress_mobilePhone, status_statusType
FROM [dbo].[paylocity_ev_employees_detail]
WHERE custom_Uber_ID = ?
"""

cursor.execute(employee_lookup, (driver_uuid,))
row = cursor.fetchone()




In [6]:
import requests
# ── 3) Lookup Employee ─────────────────────────────────────
employee_lookup = f"""
SELECT TOP 1 
    firstName, LastName, homeAddress_mobilePhone, status_statusType
FROM [dbo].[paylocity_ev_employees_detail]
WHERE custom_Uber_ID = '{driver_uuid}'
"""

cursor.execute(employee_lookup)
row = cursor.fetchone()

if not row:
    print("⚠️ No employee found for that Uber ID.")
    exit()

first_name, last_name, phone, status = row

# ── 4) Clean & Format Phone Number ─────────────────────────
if phone:
    phone = (
        phone.replace(" ", "")
        .replace("-", "")
        .replace("(", "")
        .replace(")", "")
    )
    if not phone.startswith("+1"):
        phone = "+1" + phone
else:
    print("⚠️ Missing phone number for employee.")
    exit()

# ── 5) Build Message Text ──────────────────────────────────
message_text = (
    f"Hi {first_name}, this is Tower Mobility Support.\n\n"
    f"We’re notifying you that a traffic ticket has been issued and needs to be paid.\n\n"
    f"📅 Date: {date}\n"
    f"🚗 License Plate: {license_plate}\n"
    f"📄 Ticket No: {ticket_no}\n"
    f"⚠️ Violation: {violation}\n"
    f"💰 Fine Amount: {fine_penalties}\n"
    f"⏰ Time: {time}\n"
    f"📍 Location: {location}\n\n"
    "Please make sure this ticket is paid as soon as possible to avoid additional penalties. "
    "If you’ve already taken care of it, kindly reply to this message with a confirmation or proof of payment."
)

# ── 6) Send SMS via Dialpad API ────────────────────────────
sms_url = f"https://dialpad.com/api/v2/sms?apikey={dialpad_token}"

phone = "+17815021638"
payload = {
    "infer_country_code": False,
    "to_numbers": [f"{phone}"],
    "from_number": "+13234021120",
    "text": f"This is to test Avva.0 - Tower Mobility Automated Vehicle Violation Assistant\n {message_text}"
}

headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

try:
    response = requests.post(sms_url, json=payload, headers=headers)
    response.raise_for_status()  # Raise an error for bad status codes
    print("📤 SMS sent successfully!")
    print("Response:", response.json())
except requests.exceptions.RequestException as e:
    print(f"⚠️ Failed to send SMS: {e}")
    if response is not None:
        print("Response content:", response.text)

# ── 7) Close Connection ────────────────────────────────────
cursor.close()
conn.close()


📤 SMS sent successfully!
Response: {'contact_id': '5797713501437952', 'created_date': '2025-12-05T05:14:10.080055', 'device_type': 'public_api', 'direction': 'outbound', 'from_number': '+13234021120', 'id': '6171813498331136', 'message_status': 'pending', 'target_id': '4648966761660416', 'target_type': 'department', 'text': 'This is to test Avva.0 - Tower Mobility Automated Vehicle Violation Assistant\n Hi cristina, this is Tower Mobility Support.\n\nWe’re notifying you that a traffic ticket has been issued and needs to be paid.\n\n📅 Date: 07/02/25\n🚗 License Plate: 27472Y3\n📄 Ticket No: 2011638297\n⚠️ Violation: BUS ZONE\n💰 Fine Amount: $406.00\n⏰ Time: 06:47 AM\n📍 Location: SB GRAND AVE @ 3RD S\n\nPlease make sure this ticket is paid as soon as possible to avoid additional penalties. If you’ve already taken care of it, kindly reply to this message with a confirmation or proof of payment.', 'to_numbers': ['+17815021638']}
